In [ ]:
import subprocess
import sys
import os

import uuid
from PIL import Image

import ntpath
import time

import random

In [ ]:
# util para instalação de pacotes

def instala_pacote(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

In [ ]:
# trabalhar com imagem

def carrega_imagem(image_path):
    im = Image.open(image_path)
    return im

def converte_imagem_para_jpeg(infile):
    f, e = os.path.splitext(infile)
    outfile = f + '.jpg'
    im = Image.open(infile)
    rgb_im = im.convert('RGB')
    rgb_im.save(outfile)

def exibe_imagem(im):
    im.show()

def salva_imagem(im, filename):
    im.save(filename)

def redimenciona_imagem(im, filename):
    im_resized = im.resize((224, 224))
    salva_imagem(im_resized, filename)

def copia_e_transforma_imagem(im, filename):
    diretorio = obtem_diretorio_arquivo(filename)
    arquivo = obtem_nome_arquivo(filename)
    nome_arquivo = os.path.splitext(arquivo)[0]
    extensao = os.path.splitext(arquivo)[1]
    flr = diretorio+'/'+nome_arquivo+'_FLIP_LEFT_RIGHT'+extensao
    im_flipped = im.transpose(Image.FLIP_LEFT_RIGHT)
    salva_imagem(im_flipped, flr)
    ftb = diretorio+'/'+nome_arquivo+'FLIP_TOP_BOTTOM'+extensao
    salva_imagem(im.transpose(Image.FLIP_TOP_BOTTOM), ftb)
    r90 = diretorio+'/'+nome_arquivo+'ROTATE_90'+extensao
    salva_imagem(im.transpose(Image.ROTATE_90), r90)
    r180 = diretorio+'/'+nome_arquivo+'ROTATE_180'+extensao
    salva_imagem(im.transpose(Image.ROTATE_180), r180)
    r270 = diretorio+'/'+nome_arquivo+'ROTATE_270'+extensao
    salva_imagem( im.transpose(Image.ROTATE_270), r270)

In [ ]:
# manipula arquivo

def obtem_nome_arquivo(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def obtem_diretorio_arquivo(path):
    return os.path.dirname(path)

def novo_id():
    uid = str(uuid.uuid1())
    return uid

def renomeia_arquivo(path, sufixo):
    original_path = path
    nome_arquivo=obtem_nome_arquivo(path)
    sem_extensao=os.path.splitext(nome_arquivo)[0]
    h = novo_id()
    novo_nome= '{}-{}'.format(sufixo, h)
    path = path.replace(sem_extensao, novo_nome)
    try:
        os.rename(original_path, path)
    except FileNotFoundError:
        print("não foi possivel mover "+novo_nome)

def lista_imagens(path):
    arquivos = sorted([path+'/'+file for file in os.listdir(path)])
    return arquivos

def extrai_imagens_para_teste(lista_imagens, percentual=0.1):
    total=len(lista_imagens)
    total_teste = int(total*percentual)
    print(str(total_teste)+"imagens para teste")
    if(total_teste==0):
        total_teste=1
    test_list = []
    for i in range(0,total_teste):
        test_list.append(random.choice(lista_imagens))
    return test_list

def move_arquivos_teste(lista_arquivos, path_test):
    for arquivo in lista_arquivos:
        nome_arquivo = obtem_nome_arquivo(arquivo)
        test = path_test+"/"+nome_arquivo
        print("mv "+arquivo+" "+test)
        try:
            os.rename(arquivo, test)
        except FileNotFoundError:
            print("não foi possivel mover "+test)


In [ ]:

def criar_dataset(images_path, teste_path, expande_dataset=False):
    # padroniza o nome dos arquivos
    lista_arquivos = lista_imagens(images_path)
    for arquivo in lista_arquivos:
        renomeia_arquivo(arquivo, 'amostra')
    print("arquivos renomeados") 

    # padroniza imagens para .jpg
    lista_arquivos = lista_imagens(images_path)
    for arquivo in lista_arquivos:
        if (arquivo.endswith('.png') or arquivo.endswith('.jpeg')):
            converte_imagem_para_jpeg(arquivo)
            os.remove(arquivo)
    print("extensão corrigida") 

    # Para cada imagem, aplica o redimencionamento
    for arquivo in lista_arquivos:
        im = carrega_imagem(arquivo)
        redimenciona_imagem(im, arquivo)
    print("arquivos redimencionados") 

    # Salva arquivos para teste
    dados_teste = extrai_imagens_para_teste(lista_arquivos, 0.2)
    print("arquivos para teste") 
    print(dados_teste) 

    move_arquivos_teste(dados_teste, teste_path)
    print("arquivos de teste reservados") 

    # resetar lista de imagens
    lista_arquivos = lista_imagens(images_path)

    # Para cada imagem, cria uma copia girando 90º, 180º, 270º, e imagem espelhada
    if(expande_dataset):
        for arquivo in lista_arquivos:
            im = carrega_imagem(arquivo)
            copia_e_transforma_imagem(im, arquivo)
        print("transformações realizadas") 


In [ ]:
images_path='/home/vagrant/workspace/images/lagarta'
teste_path='/home/vagrant/workspace/images/test_images/lagarta'
criar_dataset(images_path, teste_path)

images_path='/home/vagrant/workspace/images/percevejo_verde'
teste_path='/home/vagrant/workspace/images/test_images/percevejo_verde'
criar_dataset(images_path, teste_path)

images_path='/home/vagrant/workspace/images/percevejo_marrom'
teste_path='/home/vagrant/workspace/images/test_images/percevejo_marrom'
criar_dataset(images_path, teste_path)

images_path='/home/vagrant/workspace/images/percevejo_pequeno'
teste_path='/home/vagrant/workspace/images/test_images/percevejo_pequeno'
criar_dataset(images_path, teste_path)


images_path='/home/vagrant/workspace/images/negativo'
teste_path='/home/vagrant/workspace/images/test_images/negativo'
criar_dataset(images_path, teste_path)